In [21]:
years = list(range(1991, 2025))

In [9]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

In [17]:
import requests

for year in years:
    url = url_start.format(year)
    data = requests.get(url)
    with open("mvp/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [10]:
from bs4 import BeautifulSoup
import pandas as pd

In [22]:
dfs = []
for year in years:
    with open("mvp/{}.html".format(year)) as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find("tr", class_="over_header").decompose()
    mvp_table = soup.find(id="mvp")
    mvp = pd.read_html(str(mvp_table))[0]
    mvp["Year"] = year
    dfs.append(mvp)

In [23]:
mvps = pd.concat(dfs)

In [24]:
mvps.to_csv("mvps.csv")

In [ ]:
import time

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [ ]:
service = Service("/usr/local/bin/chromedriver-mac-x64/chromedriver")
driver = webdriver.Chrome(service=service)

In [ ]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
for year in years:
    url = player_stats_url.format(year)
    driver.get(url)
    driver.execute_script("window.scrollTo(1, 10000)")
    time.sleep(2)
    
    html = driver.page_source
    
    with open("player/{}.html".format(year), "w+") as f:
        f.write(html)

In [ ]:
dfs = []
for year in years:
    with open("player/{}.html".format(year)) as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find("tr", class_="thead").decompose()
    player_table = soup.find(id="per_game_stats")
    player = pd.read_html(str(player_table))[0]
    player["Year"] = year
    dfs.append(player)

In [ ]:
players = pd.concat(dfs)

In [ ]:
players

In [ ]:
players.to_csv("players.csv")

In [ ]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [ ]:
for year in years:
    url = team_stats_url.format(year)
    data = requests.get(url)

    with open("team/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [ ]:
dfs = []
for year in years:
    with open("team/{}.html".format(year)) as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    thead_rows = soup.find_all("tr", class_="thead")
    
    for thead in thead_rows:
        thead.decompose()
    
    eastern_conference_table = soup.find(id="divs_standings_E")
    team = pd.read_html(str(eastern_conference_table))[0]
    team["Year"] = year
    team["Team"] = team["Eastern Conference"]
    del team["Eastern Conference"]
    dfs.append(team)
    
    western_conference_table = soup.find(id="divs_standings_W")
    team = pd.read_html(str(western_conference_table))[0]
    team["Year"] = year
    team["Team"] = team["Western Conference"]
    del team["Western Conference"]
    dfs.append(team)

In [ ]:
teams = pd.concat(dfs)

In [ ]:
teams.to_csv("teams.csv")